In [ ]:
import bs4 as bs
import pandas as pd
import numpy as np
import random
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver import ActionChains
from selenium.webdriver import ChromeOptions
from subprocess import call
from keyboard import press
from tqdm import tqdm
from dalab import to_pickle
from dalab import read_pickle

In [ ]:
real_phones = read_pickle('data/phone_lists/real_phones.pickle')
fake_phones = read_pickle('data/phone_lists/fake_phones.pickle')

In [ ]:
# emojis = ['🔥','😀','💥','🎧','🎸','✌','🙌','😄','🙂','🙃']
emojis = [':)', ':D', '!!']

msg_1 = 'Oi, td bem? Meu nome é Layane, to divulgando o novo single dos meninos do Passageiro, dá uma olhada, ve o que vc achou e me conta depois!'
msg_2 = 'Oie, tudo joia? Eu sou a Layane, queria te mostrar a nova música do Passageiro, os meninos são MUITO BONS, aqui de Uberlândia, clica pra ver!'
msg_3 = 'E aí, tá joia? Só queria te contar que o duo Passageiro lançou uma música muito massa! Quer ver? Clica aí no link... Já está no Spotify e no YouTube! Muito obrigada pela atenção' 

phones = read_pickle('data/phone_lists/34988.pickle')
phones = [phone for phone in phones if phone not in real_phones and phone not in fake_phones]

adlink = 'https://www.youtube.com/watch?v=DyBvq1x0qwY&list=RDEMQpWOLGqXNbM7x1yAl8Rnqw&start_radio=1'
msg_list = [msg_1, msg_2, msg_3]

In [ ]:
def about(n):
    s = n/10
    n = np.random.normal(n, scale=s)
    return n

In [ ]:
load_qr = 15
load_pg = 6

next_message = about(10)
next_contact = about(120)

In [ ]:
def open_whasapp(headless=False, phone='5534988722041'):
    url = 'https://api.whatsapp.com/send?phone=' + phone + '&text='
    options = ChromeOptions()
    driver = webdriver.Chrome(options=options)
    driver.get(url)
    time.sleep(load_pg)

    chains = ActionChains(driver)
    hover = chains.move_to_element(driver.find_element_by_id("action-button"))
    hover.click().perform()
    
    print('Connected to Whatsapp, please insert QR code.')
    time.sleep(load_qr)
    return driver

In [ ]:
def create_messages(msg_list=msg_list, adlink=adlink, emojis=emojis):
    msg = random.choice(msg_list)
    emoji = random.choice(emojis)
    msg = msg + ' ' + emoji
    messages = [msg, adlink]
    return messages

In [ ]:
def send_messages(driver, phones, msg_list, adlink):
    real_phones = []
    fake_phones = []
    for phone in phones:
        try:
            messages = create_messages(msg_list, adlink)
            url = 'https://api.whatsapp.com/send?phone=' + phone + '&text='
            driver.get(url)
            time.sleep(load_pg)
            press('enter')
            press('enter')
            press('enter')
            time.sleep(load_pg)

            chains = ActionChains(driver)
            hover = chains.move_to_element(driver.find_element_by_id("action-button"))
            hover.click().perform()
            hover.click().perform()
            hover.click().perform()
            time.sleep(load_pg)

            chains = ActionChains(driver)
            hover = chains.move_to_element(driver.find_element_by_class_name("_1Plpp"))
            hover.click().perform()
            hover.reset_actions()

            for msg in messages:
                hover.send_keys(msg).perform()
                time.sleep(load_pg)
                hover.reset_actions()

                hover.send_keys(Keys.ENTER).perform()
                time.sleep(next_message)

            time.sleep(next_contact)
            real_phones.append(phone)
            to_pickle(real_phones, 'data/phone_lists/real_phones')
            
        except: 
            print('Could not send message.')
            fake_phones.append(phone)
            to_pickle(fake_phones, 'data/phone_lists/fake_phones')
                
            

In [ ]:
driver = open_whasapp()

In [ ]:
send_messages(driver, phones, msg_list, adlink)

In [ ]:
def generate_phone(n_digits=9, ddi='55', ddd='34', first_numbers='988'):
    digits = [0,1,2,3,4,5,6,7,8,9]
    phone = [i for i in ddi + ddd + first_numbers]
    n = n_digits - len(first_numbers)
    for i in range(n):
        phone.append(str(random.choice(digits)))
    return ''.join(phone)

In [ ]:
def generate_phone_list(n):
    phone_list = []
    for _ in tqdm(range(n)):
        phone_list.append(generate_phone())
    phone_list = list(set(phone_list))
    return phone_list

In [ ]:
# phone_list = generate_phone_list(20000000)
# to_pickle(phone_list, 'data/phone_lists/34988')